# สวัสดีครับ วันนี้เราก็จะมาทำ GO enrichment ด้วย Fisher's Exact Test กันนะครับผม

อันนี้ก็ขอปูพื้นกันก่อนน๊าาาาาา

1. Gene Ontology
http://geneontology.org/docs/ontology-documentation/

2. Fisher exact test
http://project.astyleplus.net/lesson3_3.html

3. GO enrichment with Fisher Exact Test
https://www.nature.com/articles/srep04191
https://www.pathwaycommons.org/guide/primers/statistics/fishers_exact_test/

# เริ่มรันโค๊ดกันได้เลย

In [10]:
##Import Required Modules
from string import*
import os
import numpy as np
import scipy.stats as stats

In [11]:
oddsratio, pvalue = stats.fisher_exact([[900, 1000], [10000, 5000000]])

In [12]:
oddsratio

450.0

In [14]:
float(pvalue)

0.0

In [3]:
##[0]Create function
def countNoGene(goId,data):
    noGene = 0
    for i in data:
        iList = i.split("\t")
        term = ";".join(iList[20:23])##check GO with 3 categories
        if goId in term:
            noGene = noGene + 1
    return noGene

##[1]Load input file and creating variable
path = "C:\\Users\\wutto\\Desktop\\ML_aspergillus_aculeatus\\goEnrichment\\input\\"
os.chdir(path)
fileList = os.listdir(path)

dataAll = open("C:\\Users\\wutto\\Desktop\\ML_aspergillus_aculeatus\\goEnrichment\\4_ortholog_gene_with_selected_function_only_heatMap.txt",'r').readlines()
header = "fileName\tGO_term_and_ID\tA\tB\tC\tD\toddRatio\tP-value\tcategory\tlist_sig_genes\n"
del dataAll[0]##delete header

saveFile = open("C:\\Users\\wutto\\Desktop\\ML_aspergillus_aculeatus\\goEnrichment\\GO_enrichment_7_cluster_with_gene_in_heatmap.txt","w")
saveFile.write(header)

saveFileFilter = open("C:\\Users\\wutto\\Desktop\\ML_aspergillus_aculeatus\\goEnrichment\\GO_enrichment_7_cluster_with_gene_in_heatmap_filter.txt","w")
saveFileFilter.write(header)


"""create GO term and category dictionaries"""
goCat = {}
for i in dataAll:
    iList = i.split("\t")
    
    if "GO:" in iList[20]:
        for key in iList[20].split("; "):
            goCat[key] = "cellular_component"

    if "GO:" in iList[21]:
        for key in iList[21].split("; "):
            goCat[key] = "biological_process"

    if "GO:" in iList[22]:
        for key in iList[22].replace("\n","").split("; "):
            goCat[key] = "molecular_function"

"""Calculate Chi-Square"""    
for fileName in fileList:
    print (fileName)
    
    """Loading DEG list"""
    sigGene = open(fileName,'r').readlines()

    for i in range(0,len(sigGene)):
        sigGene[i] = sigGene[i].strip()
        
    data = []
    dataNoSig = []
    for i in dataAll:
        if (i.split("\t")[1] in sigGene):
            data.append(i)
        else:
            dataNoSig.append(i)

    """Adding data"""
    go = {}
    for i in data:
        iList = i.split("\t")
        #cellular_component
        if "GO:" in iList[20]:
            member = iList[1]
            for key in iList[20].split("; "):
                if key not in go:
                    value = []
                else:
                    value = go[key]
                value.append(member)##adding value into result
                go[key] = value

        #biological_process
        if "GO:" in iList[21]:
            member = iList[1]
            for key in iList[21].split("; "):
                if key not in go:
                    value = []
                else:
                    value = go[key]
                value.append(member)##adding value into result
                go[key] = value

        #molecular_function
        if "GO:" in iList[22]:
            member = iList[1]
            for key in iList[22].replace("\n","").split("; "):
                if key not in go:
                    value = []
                else:
                    value = go[key]
                value.append(member)##adding value into result
                go[key] = value

        
    """Count Strain"""
    for goId in go.keys():
        a = len(go[goId])
        b = countNoGene(goId,dataNoSig)
        c = len(sigGene) - a
        d = countNoGene("GO:",dataNoSig) - b
        oddsratio, pvalue = stats.fisher_exact([[a, b], [c, d]])
        #print fileName,"\t",goId,"\t",a,"\t",b,"\t",c,"\t",d,"\t",oddsratio,"\t",pvalue,"\t",goTerm[goId],"\t",goCat[goId],"\t",join(go[goId],";")
        #dataRecord = fileName+"\t"+goId+"\t"+str(a)+"\t"+str(b)+"\t"+str(c)+"\t"+str(d)+"\t"+str(oddsratio)+"\t"+str(pvalue)+"\t"+goTerm[goId]+"\t"+goCat[goId]+"\t"+join(go[goId],";")
        dataRecord = fileName.split("_")[1]+"\t"+goId+"\t"+str(a)+"\t"+str(b)+"\t"+str(c)+"\t"+str(d)+"\t"+str(oddsratio)+"\t"+str(pvalue)+"\t"+goCat[goId]+"\t"+";".join(go[goId])
        saveFile.write(dataRecord)
        saveFile.write("\n")
        
        ##write filtered data into an output file
        if (pvalue < 0.05) and (str(oddsratio) != "inf"):
            saveFileFilter.write(dataRecord)
            saveFileFilter.write("\n")
            
    # [a | b ]
    # --------
    # [c | d ]

"""
***เลือกเฉพาะยีนที่มี GO annotation


             NO.genes in current cluster      genes in other cluster 
             |==========================|=============================|
target GO ID |_______________a__________|______________b______________|
other GO ID  |               c          |              d              |
             |==========================|=============================|
    
"""
saveFile.close()
saveFileFilter.close()

TPM_cluster1_in_heatmap_13July.txt
TPM_cluster2_in_heatmap_13July.txt
TPM_cluster3_in_heatmap_13July.txt
TPM_cluster4_in_heatmap_13July.txt
TPM_cluster5_in_heatmap_13July.txt
TPM_cluster6_in_heatmap_13July.txt
TPM_cluster7_in_heatmap_13July.txt


In [13]:
##calculate distribution
b = open("C:\\Users\\wutto\\Desktop\\ML_aspergillus_aculeatus\\goEnrichment\\GO_enrichment_7_cluster_with_gene_in_heatmap_filter.txt",'r').readlines()
del b[0]##delete header

go = {}
cluster = {}
for i in b:
    go[i.split("\t")[1]] = ""
    cluster[i.split("\t")[0]] = ""

goList = list(go.keys())
goList.sort()

clusterList = list(cluster.keys())


saveFile = open("C:\\Users\\wutto\\Desktop\\ML_aspergillus_aculeatus\\goEnrichment\\GO_enrichment_7_cluster_with_gene_in_heatmap_filter_summary.txt","w")
header = "GO_term\t"+"\t".join(clusterList)+"\n"
saveFile.write(header)

for goTerm in goList:
    record = []
    record.append(goTerm)
    for clusterId in cluster:
        num = "0"
        for i in b:
            if (clusterId == i.split("\t")[0]) and (goTerm == i.split("\t")[1]):
                num = i.split("\t")[2]
        record.append(num)
    saveFile.write("\t".join(record))
    saveFile.write("\n")
                
saveFile.close()